# Test

In [ ]:
import numpy as np 
import pandas as pd 

def objective_function(x, test, prob):
    a, b, c = x
    return np.sum((a * test[0] + b * test[1] + c * test[2] - prob) ** 2)

def constraint(x):
    return np.sum(x) - 1

# create a random np vector of probabilities
prob = np.random.random(500)

test = np.random.random((3,500))

x0 = np.array([1/3, 1/3, 1/3])

In [ ]:
objective_function(x0, test, prob)

In [ ]:
import numpy as np
from scipy.optimize import minimize

con = {'type': 'eq', 'fun': constraint}
bounds = [(0, 1), (0, 1), (0, 1)]  # a, b, c >= 0 and <= 1

result = minimize(objective_function, x0, args=(test, prob), bounds=bounds, constraints=con)

print("Optimal values:")
print("a =", result.x[0])
print("b =", result.x[1])
print("c =", result.x[2])
print("Objective function value:", result.fun)

### Hamming Distance

In [ ]:
import os
import pandas as pd
import numpy as np

path = '/data1/malto/shroom'

results1 = pd.read_csv(path + '/baseline_val.csv')
results2 = pd.read_csv(path + '/crlft_val.csv')
results3 = pd.read_csv(path + '/sequential_nogpt_aware.csv')

In [ ]:
r1 = 1 - results1['baseline'].to_numpy()
r2 = results2['deberta-xlarge-mnli_0.01_frozen_22_24_1_0.772_63_sequential'].to_numpy()
r3 = 1 - results3['sequential_nogpt_deberta_xlarge'].to_numpy()

test = np.stack((r1, r2, r3), axis=0).mean(axis=0)
test.shape

In [ ]:
labels = pd.read_json(path + '/val.model-aware.json', lines=True)['p(Hallucination)'].to_numpy()
labels.shape

In [ ]:
predictions = np.where(test > 0.5, 1, 0)
labels = np.where(labels > 0.5, 1, 0)
average_wrongs = np.logical_xor(predictions, labels).sum()
average_wrongs

In [ ]:
labels1 = labels[:len(labels)//2]
labels2 = labels[len(labels)//2:]

r1a = r1[:len(r1)//2]
r1b = r1[len(r1)//2:]

r2a = r2[:len(r2)//2]
r2b = r2[len(r2)//2:]

r3a = r3[:len(r3)//2]
r3b = r3[len(r3)//2:]

test1 = np.stack((r1a, r2a, r3a), axis=0)
test2 = np.stack((r1b, r2b, r3b), axis=0)

In [ ]:
import numpy as np 
import pandas as pd 

def objective_function(x, test, prob):
    a, b, c = x
    return np.sum((a * test[0] + b * test[1] + c * test[2] - prob) ** 2)

def constraint(x):
    return np.sum(x) - 1

# create a random np vector of probabilities
prob = labels1 

test = np.stack((r1, r2, r3), axis=0)

x0 = np.array([1/3, 1/3, 1/3])

In [ ]:
objective_function(x0, test1, labels1), objective_function(x0, test2, labels2)

In [ ]:
import numpy as np
from scipy.optimize import minimize

con = {'type': 'eq', 'fun': constraint}
bounds = [(0, 1), (0, 1), (0, 1)]  # a, b, c >= 0 and <= 1

result = minimize(objective_function, x0, args=(test, labels), bounds=bounds, constraints=con)

print("Optimal values:")
print("a =", result.x[0])
print("b =", result.x[1])
print("c =", result.x[2])
print("Objective function value:", result.fun)

In [ ]:
objective_function(result.x, test2, labels2)

In [ ]:
import os
import pandas as pd
import numpy as np

path = '/data1/malto/shroom'

results1 = pd.read_csv(path + '/baseline.csv')
results2 = pd.read_csv(path + '/crlft.csv')
results3 = pd.read_csv(path + '/sequential_nogpt.csv')

In [ ]:
results2

In [ ]:
r1 = 1 - results1['baseline'].to_numpy()
r2 = results2['crlft_alldata_deberta_xlarge_forzen_22_sequential'].to_numpy()
r3 = 1 - results3['sequential_nogpt_deberta_xlarge'].to_numpy()

test = np.stack((r1, r2, r3), axis=0).mean(axis=0)
test.shape

In [ ]:
r1_new = result.x[0] * r1
r2_new = result.x[1] * r2
r3_new = result.x[2] * r3

res = r1_new + r2_new + r3_new
res

In [ ]:
panda = pd.DataFrame(res, columns=['p(Hallucination)'])
panda.to_csv(path + '/lp_ensemble.csv', index=False)

In [ ]:
res = pd.read_csv(path + '/lp_ensemble.csv')['p(Hallucination)'].to_numpy()
res

In [ ]:
labels_sequential = np.where(r3 > 0.5, 1, 0)
labels_ensemble = np.where(res > 0.5, 1, 0)
differences = np.logical_xor(labels_sequential, labels_ensemble).sum()
differences

# Paper evaluation of ensembles 

In [ ]:
# read files in paper_results directory 
import os
import pandas as pd
import numpy as np
from scipy.optimize import minimize


n = 4
use_mnli = True
result_path = 'paper_results_mnli/' if use_mnli else 'paper_results/'
path = '/data1/malto/shroom'


baseline_test = pd.read_csv(result_path + '/baseline_test' + str(n) + '.csv')['baseline'].to_numpy()
baseline_val = pd.read_csv(result_path + '/baseline_val' + str(n) + '.csv')['baseline'].to_numpy()
crlft_test = pd.read_csv(result_path + '/crlft_test' + str(n) + '.csv')['crlft'].to_numpy()
crlft_val = pd.read_csv(result_path + '/crlft_val' + str(n) + '.csv')['crlft'].to_numpy()
sequential_test = pd.read_csv(result_path + '/sequential_test' + str(n) + '.csv')['sequential'].to_numpy() 
sequential_val = pd.read_csv(result_path + '/sequential_val' + str(n) + '.csv')['sequential'].to_numpy()

labels_val = pd.read_json(path + '/val.model-aware.json', lines=True)['p(Hallucination)'].to_numpy()

In [ ]:
# baseline: da ribaltare solo test 
# crlft: da ribaltare entrambi 

In [ ]:
predictions = np.where(sequential_val > 0.5, 1, 0)
labels = np.where(labels_val > 0.5, 1, 0)
print(np.sum(predictions == labels))

In [ ]:
# read test.model-agnostic.json 
from datasets import load_dataset, concatenate_datasets, DatasetDict

test = load_dataset("json", data_files=["test.model-agnostic.json"])
ground_truth_test = test['train']['label']
ground_truth_test = [1 if x == 'Hallucination' else 0 for x in ground_truth_test]
ground_truth_test = np.array(ground_truth_test)
ground_truth_test

In [ ]:
predictions = np.where(sequential_test > 0.5, 1, 0)
labels = np.where(ground_truth_test > 0.5, 1, 0)
print(np.sum(predictions == labels))

In [ ]:
predictions.shape, labels.shape

3-3

In [ ]:
ground_truth = labels_val

In [ ]:
# optimization problem between the three models
import numpy as np

def objective_function(x, res, ground_truth):
    a, b, c = x
    return np.sum((a * res[0] + b * res[1] + c * res[2] - ground_truth) ** 2)

def constraint(x):
    return np.sum(x) - 1

con = {'type': 'eq', 'fun': constraint}
bounds = [(0, 1), (0, 1), (0, 1)]  # a, b, c >= 0 and <= 1
x0 = np.array([1/3, 1/3, 1/3])

In [ ]:
res = np.stack((baseline_val, sequential_val, crlft_val), axis=0)

In [ ]:
result = minimize(objective_function, x0, args=(res, ground_truth), bounds=bounds, constraints=con)

print("Optimal values:")
print("a =", result.x[0])
print("b =", result.x[1])
print("c =", result.x[2])
print("Objective function value:", result.fun)

In [ ]:
r1_new = result.x[0] * baseline_test
r2_new = result.x[1] * sequential_test
r3_new = result.x[2] * crlft_test

res = r1_new + r2_new + r3_new
res

In [ ]:
panda = pd.DataFrame(res, columns=['p(Hallucination)'])
panda.to_csv(result_path + f'/ensemble_full{n}.csv', index=False)

for 2-2

In [ ]:
# attach together crlft_val and baseline_val in a (2, 500) array
res1 = np.stack((crlft_val, baseline_val), axis=0)
res1

In [ ]:
# optimization problem between the three models
import numpy as np

def objective_function(x, res, ground_truth):
    a, b = x
    return np.sum((a * res[0] + b * res[1] - ground_truth) ** 2)

def constraint(x):
    return np.sum(x) - 1

con = {'type': 'eq', 'fun': constraint}
bounds = [(0, 1), (0, 1)]  # a, b >= 0 and <= 1
x0 = np.array([1/2, 1/2])

In [ ]:
objective_function(x0, res1, ground_truth)

In [ ]:
from scipy.optimize import minimize

result = minimize(objective_function, x0, args=(res1, ground_truth), bounds=bounds, constraints=con)

print("Optimal values:")
print("a =", result.x[0])
print("b =", result.x[1])
print("Objective function value:", result.fun)

In [ ]:
r1_new = result.x[0] * crlft_test
r2_new = result.x[1] * baseline_test

res = r1_new + r2_new
res

In [ ]:
panda = pd.DataFrame(res, columns=['p(Hallucination)'])
panda.to_csv(result_path + f'/ensemble_baseline_crlft{n}.csv', index=False)

2

In [ ]:
# attach together crlft_val and baseline_val in a (2, 500) array
res1 = np.stack((crlft_val, sequential_val), axis=0)
res1

In [ ]:
# optimization problem between the three models
import numpy as np

def objective_function(x, res, ground_truth):
    a, b = x
    return np.sum((a * res[0] + b * res[1] - ground_truth) ** 2)

def constraint(x):
    return np.sum(x) - 1

con = {'type': 'eq', 'fun': constraint}
bounds = [(0, 1), (0, 1)]  # a, b >= 0 and <= 1
x0 = np.array([1/2, 1/2])

In [ ]:
objective_function(x0, res1, ground_truth)

In [ ]:
from scipy.optimize import minimize

result = minimize(objective_function, x0, args=(res1, ground_truth), bounds=bounds, constraints=con)

print("Optimal values:")
print("a =", result.x[0])
print("b =", result.x[1])
print("Objective function value:", result.fun)

In [ ]:
r1_new = result.x[0] * crlft_test
r2_new = result.x[1] * sequential_test

res = r1_new + r2_new
res

In [ ]:
panda = pd.DataFrame(res, columns=['p(Hallucination)'])
panda.to_csv(result_path + f'/ensemble_sequential_crlft{n}.csv', index=False)

3

In [ ]:
# attach together crlft_val and baseline_val in a (2, 500) array
res1 = np.stack((baseline_val, sequential_val), axis=0)
res1

In [ ]:
# optimization problem between the three models
import numpy as np

def objective_function(x, res, ground_truth):
    a, b = x
    return np.sum((a * res[0] + b * res[1] - ground_truth) ** 2)

def constraint(x):
    return np.sum(x) - 1

con = {'type': 'eq', 'fun': constraint}
bounds = [(0, 1), (0, 1)]  # a, b >= 0 and <= 1
x0 = np.array([1/2, 1/2])

In [ ]:
objective_function(x0, res1, ground_truth)

In [ ]:
from scipy.optimize import minimize

result = minimize(objective_function, x0, args=(res1, ground_truth), bounds=bounds, constraints=con)

print("Optimal values:")
print("a =", result.x[0])
print("b =", result.x[1])
print("Objective function value:", result.fun)

In [ ]:
r1_new = result.x[0] * baseline_test
r2_new = result.x[1] * sequential_test

res = r1_new + r2_new
res

In [ ]:
panda = pd.DataFrame(res, columns=['p(Hallucination)'])
panda.to_csv(result_path + f'/ensemble_sequential_baseline{n}.csv', index=False)